# <center> **Base de données (Films)**

## **Présentation**

Des fichiers csv contenant les données ont été générés lors de la phase précédente de scrapping, nous allons maintenant lire ces fichiers csv dans des dataFrame Pandas et nettoyer les données puis les insérer dans une base de données SQL.


## **Questions**

**Quelles données mettre en SQL et en NoSQL ?**<br>
SQL pour les données tabulaires, NoSQL pour les données non tabulaires ou non structurées.

**Générer automatiquement des IDs chaines de caractères ?**

**Les requêtes faites à partir de notre API doivent avoir accès uniquement à la base en lecture seule**


## **Sources**

**Neo4j pour créer des graphs ML**<br>
https://neo4j.com/docs/getting-started/appendix/tutorials/guide-import-relational-and-etl/


**Wikidata**<br>
https://query.wikidata.org/querybuilder/?uselang=fr<br>


## **Création du schéma relationnel**

![relation_schema](images/mpd.png)

Online tool: https://drawdb.vercel.app/editor<br>
https://www.mocodo.net/<br>

Lien vers mon schéma : [https://www.mocodo.net/?mcd=eNq9U0Fu2zAQvPMVe_FNKOqrb6pMNwRsKhDlBj0ZtMykBCTSoEgX6Q_6lKBP6C3-WClSkoUYPRQ1woOwyx3OzixWs9mND5rBbc9fGFPG8oykJckpLDEszz_vMV2mNMOwymnW3VO8XuN_oPxfkUzCSVsjgLetriS3Uivg4JSAipuDVLyW9vwC83kCDZctcFc5j7DWyL2ziX_tWjhqdxI_esqag9JNIwy8_oLlCl5_wwI2utIH3dWMOJrzSyuUFYbDkRvfDCphqlp8gHdzfvMVQktS4KzMC7aAgzSistrs5CG5JIo3AqU9hI91film-eY-Z7irV7o56lZEyJhEVFriz3lBcAfjVjxp8xxhfeJ5trTsAdopO9RjPEhlsMm_EJzARxojFsLRh2_ESva2nPa1KNVzbBnJ3oJGI-guZdAL_nqFGo0gGn6KdE3Ka9RgBqEC-9sH78qIkxTfg6lG-zAO2ttPoLXc7IzfY_WU9LgEXJxkUNOTJDCnl3hsuYASsxINyYXdSlt7-oMz4R_puGvBW7GLbdV-p7QVbYhiWx9L9ajD64mqIILQVd5JmI9OfRguUfguJk9d03Dz7E2Yeme_uWavuKwReihIWWIKn-LEtt24p6YW0CH82LZxo9ywTW7YpKnWoIqRDVmnxXT6VyH6A09tYpQ=
](Link Mocodo)<br>

L'utilitaire en ligne **drawdb** permet d'exporter un fichier .SQL contenant la description de notre base (tables avec les relations)

![export_SQL](images/export_SQL.png)

Après quelques modifications à ce fichier nous pouvons créer notre base de données ainsi que les tables à partir de l'invite de commandes.

<code>"C:\Program Files\MySQL\MySQL Server 8.0\bin\mysql.exe" < movies.sql -u root -p</code>



Une difficulté rencontrée est de faire apparaître les films similaires à un film dans la base de données, en effet cette situation est une relation "Many to many" où la table se référence elle-même, c'est un cas fréquent dans les réseaux sociaux où un utilisateur va avoir des amis eux-mêmes utilisateurs.<br>
Deux situations sont observées : <br>
- les relations symétriques (l'utilisateur Alice est amis avec l'utilisateur Bob implique nécessairement que Bob est ami d'Alice),
- les relations asymétriques (Alice est amis avec Bob mais Bob n'est pas forcément ami avec Alice).

Dans notre situation la relation est .....????

Pour représenter cette relation ....





**About Self many-to-many relationship**: https://stackoverflow.com/questions/17128472/many-to-many-on-same-table

Relations n-n

Remarque :
La table "reviews" sert de table de jonction entre les films et les utilisateurs, en effet la relation movies-users est une relation n-n car un utilateur peut écrire des avis pour plusieurs films et un film possède des avis de plusieurs utilisateurs.




### **Imports**

In [70]:
%reset

In [84]:
import math
import copy
import time
import re
import uuid
import json
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from unidecode import unidecode

import pymongo
import pprint
import mysql.connector
from IPython.display import display

# MongoDB / Pymongo
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

pd.set_option('display.max_rows', 10)
tqdm.pandas()

api_key = "b8dd5759"

## **Requetage de l'API IMDB**

In [122]:
def format_string(st):
    res = ''
    for c in st:
        if c.isdigit() or c.isalpha() or c.isspace():
            res += unidecode(c)
        else:
            res += ' '
    return '+'.join([word for word in res.split() if len(word) > 1])

def request_omdb_from_title(title):
    ''' Request the omdb API 
    
        return a dictionary with the information about the movie.

        Arg:
         - title: string with title of the movie we want the infos about.
    '''
    # str_title = ''.join([unidecode(c) for c in str_title if c.isdigit() or c.isalpha() or c.isspace()])
    # print(str_title)
    # title = '+'.join([item for item in str_title.split() if len(item) > 1])
    # print(title)
    url = f"https://www.omdbapi.com/?apikey={api_key}&t={format_string(title)}"
    print(url)
    r = requests.get(url)
    if r.status_code != 200:
        print(f"Response Code: {r.status_code}")
        return {}
    return json.loads(r.text)

# print(request_omdb_from_title("Moi, Christiane F., 13 ans, droguée, prostituée..."))
# print(request_omdb_from_title("Christiane droguée"))
print(request_omdb_from_title("Le Père Noël est une ordure"))
# print(request_omdb_from_title("Y a-t-il un Français Dans la Salle ?"))
# print(request_omdb_from_title("300"))


https://www.omdbapi.com/?apikey=b8dd5759&t=Le+Pere+Noel+est+une+ordure
{'Title': 'Le Pere Noel Est Une Ordure', 'Year': '2019', 'Rated': 'N/A', 'Released': 'N/A', 'Runtime': 'N/A', 'Genre': 'Short', 'Director': 'N/A', 'Writer': 'N/A', 'Actors': 'Sonia Joubert', 'Plot': 'N/A', 'Language': 'N/A', 'Country': 'N/A', 'Awards': 'N/A', 'Poster': 'N/A', 'Ratings': [], 'Metascore': 'N/A', 'imdbRating': 'N/A', 'imdbVotes': 'N/A', 'imdbID': 'tt10438376', 'Type': 'movie', 'DVD': 'N/A', 'BoxOffice': 'N/A', 'Production': 'N/A', 'Website': 'N/A', 'Response': 'True'}


## **Création de la base de données**

On lance **MySQL Shell** puis on passe en mode **SQ** avec l'instruction <code>\sql</code>

On peut ensuite lancer la création de notre base de données et des tables en lançant le script "movies.sql"

<code> "C:\Program Files\MySQL\MySQL Server 8.0\bin\mysql.exe" < movies.sql -u root -p</code><br>

Une fois la base 'movies' créée, on peut créer un connecteur sur la base.

In [72]:
cnx = mysql.connector.connect(user='root', password='admin', \
                              host = '127.0.0.1', database='movies')
cursor = cnx.cursor(buffered=True)

In [14]:
cnx.disconnect()

In [ ]:
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Create database "allocine" (or selects it if already exists)
mydb = client["allocine"]
# client.drop_database("movies")

# Create a collection "movies" (table in SQL)
col_movies = mydb["movies"]
col_movies.drop()

# Insertion of texts into MongoDB database
col_movies.insert_many(df_data.to_dict(orient='records')) # TO DO ONLY ONCE
col_movies.drop()

# Insertion of texts into MongoDB database (ALREADY DONE)
# for index, row in df_data[:20].iterrows():
#     print(index, row)
#     col_txts.insert_one({"text" : row['text'], "label" : row["label"]})

# Display some infos
# (remark: nothing is created as long as there is no data in the collection)

print(client.list_database_names())
print(mydb.list_collection_names())

for doc in list(col_movies.find().limit(5)):
    pprint.pprint(doc)

## **Remplissage de la base de données**

### **Tools**

In [28]:
def string_with_comma_to_list_of_strings(st):
    ''' Convert a string such as "['string1', 'string2' ...]" into
        a list of string ['string1', 'string2', ...]

        Return: A list of strings.

        Arg:
         - st: string with the value to split.
    '''
    if pd.isna(st):
        return []
    return [item.strip() for item in st.split(",") if len(item.strip()) > 1]

def duration_to_minutes(st):
    ''' Convert duration string into number of minutes (integer)
        1h 35min   ---->    95

        Return: Integer representing the number of minutes.
        Arg:
         - st: duration string to convert.
    '''
    if pd.isna(st) or st == '': return 0
    if 'h' in st:
        a, b = st.split('h')
        if 'min' in b:
            b = b.replace('min', '')
            return 60 * int(a) + int(b.strip())
        return 60 * int(a)
    assert 'min' in st
    st = st.replace('min', '')
    return 60 * int(st)            

def unique_values_of_columns(df_data, column):
    ''' return a list with unique values found in the column,
        values in the column are like : 'value1,value2,value3' ..... 
        so we split all values in each row of the column and stack them in a list.

        Return : A list of values.

        Args:
         - df_data: dataframe with the data to extract,
         - column: string with the name of the column we want to work with.     
    '''
    df = df_data[column]
    df.dropna(inplace = True)
    df = df.apply(string_with_comma_to_list_of_strings)
    df = df.apply(pd.Series).stack().reset_index(drop=True)
    return df.unique()

def convert_months_FR_to_EN(st):
    ''' Convert french months to english months

        Return: string with a month in english.

        Arg:
         - st: string with a french date.
    '''
    if pd.isna(st): return ''
    for month_FR, month_EN in zip(['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre'],
                                  ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']):
        if month_FR in st:
            return st.replace(month_FR, month_EN)
    print('ERROR', st)
    return st


# ------------------------ #
#                          #
#    Filling SQL tables    #
#                          #
# ------------------------ #

def generate_ID():
    return str(uuid.uuid4())

def fill_in_table(lst, table_name, field_id, field, connector, cursor):
    ''' Fill in an SQL table from a list of values.
        For each value in the list 'lst' we generate an ID and insert into the table (ID, value).

        CAREFUL: To run only once for each table, otherwise : "ERROR Duplicate entry '0' for key 'table_name.PRIMARY'"

        Return: A dictionary mapping each value of the list to an ID newly generated {value1 : ID1, value2 : ID2, ...}

        Args:
         - lst: list of values to insert into the table,
         - table_name: string with the name of the table,
         - field_id: string with the ID of the value record inserted in the table,
         - field: string of the field in the table,
         - connector: MySQL connector connected to the relevant database,
         - cursor: MySQL cursor to execute SQL statements.
    '''
    dic_return = {}
    for item in lst:
        dic_return[item] = generate_ID()
        sql = f"INSERT INTO {table_name} ({field_id}, {field}) VALUES (%s, %s)"
        val = (dic_return[item], item)
        cursor.execute(sql, val)
    connector.commit()
    return dic_return

def fill_in_pivot_table(table_name, field, lst_values, movie_id, cursor):
    ''' Fill in pivot table with couple value such (item, movie_id)
        where item is a value of lst_values.

        Args:
         - table_name: string with name of the pivot_table to fill in,
         - field: string with the name of the field (category_id, actor_id ...),
         - lst_values: list of values of the field,
         - movie_id: id of the movie to be connected
         - cursor: MySQL cursor to execute SQL statements.
    '''
    for item in lst_values:
        sql = f"INSERT INTO {table_name} ({field}, movie_id) VALUES (%s, %s)"
        val = (item, movie_id)
        cursor.execute(sql, val)

### **Lecture des données à partir des CSV**

In [34]:
ds_categories = pd.read_csv('csv/categories.csv', delimiter = ',')
ds_categories = ds_categories[ds_categories.columns[0]]
ds_countries = pd.read_csv('csv/countries.csv', delimiter = ',')
ds_countries = ds_countries[ds_countries.columns[0]]

df_movies = pd.read_csv('csv/movies_year_1982.csv', delimiter = ',')
# df_movies = pd.read_csv('csv/movies_decade_80.csv', delimiter = ',')
#df_movies = pd.read_csv('csv/movies_year_1960_to_1990.csv', delimiter = ',')

print("Categories :", ds_categories.shape)
print("Countries :", ds_countries.shape)
print("movies :", df_movies.shape)

Categories : (37,)
Countries : (104,)
movies : (81, 15)


### **Remplissage des tables**

In [ ]:
# ------------------------------ #
#    Fill in categories table    #
# ------------------------------ #
# Create from df_movies, not from df_categories
dict_category_id = fill_in_table(ds_categories.values.tolist(), 'categories', 'category_id', 'category', cnx, cursor)

# ------------------------------ #
#    Fill in countries table     #
# ------------------------------ #
# Create from df_movies, not from df_countries
dict_country_id = fill_in_table(ds_countries.values.tolist(), 'countries', 'country_id', 'country', cnx, cursor)

# ------------------------------ #
#    Fill in directors table     #
# ------------------------------ #
lst_directors = unique_values_of_columns(df_movies, 'directors')
dict_director_id = fill_in_table(lst_directors, 'directors', 'director_id', 'director_name', cnx, cursor)

# ------------------------------ #
#      Fill in actors table      #
# ------------------------------ #
lst_actors = unique_values_of_columns(df_movies, 'actors')
dict_actor_id = fill_in_table(lst_actors, 'actors', 'actor_id', 'actor_name', cnx, cursor)

# ------------------------------ #
#     Fill in composers table    #
# ------------------------------ #
lst_composers = unique_values_of_columns(df_movies, 'composers')
dict_composer_id = fill_in_table(lst_composers, 'composers', 'composer_id', 'composer_name', cnx, cursor)

### **Remplissage de la table des films**

In [35]:
# --------------------------------------------------------
#
#  Converting some columns of "df_movies" into the appropriate format:
#   - correct date format
#   - duration un minutes
#   - IDs of the categories
#   - IDs of the directors / actors / composers
#
# --------------------------------------------------------

df_movies_formatted = df_movies.copy()
df_movies_formatted['categories'] = df_movies_formatted['categories'].map(string_with_comma_to_list_of_strings)
df_movies_formatted['categories'] = df_movies_formatted['categories'].apply(lambda lst : [dict_category_id[k] for k in lst])

df_movies_formatted['countries'] = df_movies_formatted['countries'].map(string_with_comma_to_list_of_strings)
df_movies_formatted['countries'] = df_movies_formatted['countries'].apply(lambda lst : [dict_country_id[k] for k in lst])

df_movies_formatted['directors'] = df_movies_formatted['directors'].map(string_with_comma_to_list_of_strings)
df_movies_formatted['directors'] = df_movies_formatted['directors'].apply(lambda lst : [dict_director_id[k] for k in lst])

df_movies_formatted['actors'] = df_movies_formatted['actors'].apply(string_with_comma_to_list_of_strings)
df_movies_formatted['actors'] = df_movies_formatted['actors'].apply(lambda lst : [dict_actor_id[k] for k in lst])

df_movies_formatted['composers'] = df_movies_formatted['composers'].apply(string_with_comma_to_list_of_strings)
df_movies_formatted['composers'] = df_movies_formatted['composers'].apply(lambda lst : [dict_composer_id[k] for k in lst])

df_movies_formatted['duration'] = df_movies_formatted['duration'].apply(duration_to_minutes)

df_movies_formatted['date'] = df_movies_formatted['date'].apply(convert_months_FR_to_EN)
df_movies_formatted['date'] = pd.to_datetime(df_movies_formatted['date'], format='mixed')

df_movies_formatted['notes'] = df_movies_formatted['notes'].apply(int)
df_movies_formatted['reviews'] = df_movies_formatted['reviews'].astype(int)
df_movies_formatted['star_rating'] = df_movies_formatted['star_rating'].apply(lambda x : float(x.replace(',', '.')))

In [ ]:
for movie in df_movies_formatted.itertuples():

    # ----------- #
    #    infos    #
    # ----------- #

    info_id = generate_ID()
    sql = "INSERT INTO infos (info_id, summary, url_thumbnail) VALUES (%s, %s, %s)"
    val = (info_id, 
           movie[12],  # summary 
           movie[13])  # url_thumbnail
    cursor.execute(sql, val)

    # ----------- #
    #    movies   #
    # ----------- #

    movie_id = generate_ID()

    sql = "INSERT INTO movies (movie_id, title, release_date, duration, nb_notes, \
                               nb_reviews, info_id, star_rating) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    val = (movie_id, 
           movie[1], # title
           movie[2], # release_date
           movie[3], # duration
           movie[7], # nb_notes
           movie[8], # nb_reviews
           info_id,  
           movie[6]) # star_rating
    cursor.execute(sql, val)

    # -------------------- #
    #     Pivot tables     #   
    # -------------------- #
    #    category_movie    #
    #    country_movie     #
    #    director_movie    #
    #    actor_movie       #
    #    composer_movie    #
    # -------------------- #

    # Fill in pivot table: category_movie
    lst_categories = movie[4]
    fill_in_pivot_table('category_movie', 'category_id', lst_categories, movie_id, cursor)

    # Fill in pivot table: country_movie
    lst_countries = movie[5]
    fill_in_pivot_table('country_movie', 'country_id', lst_countries, movie_id, cursor)

    # Fill in pivot table: director_movie
    lst_directors = movie[9]
    fill_in_pivot_table('director_movie', 'director_id', lst_directors, movie_id, cursor)

    # Fill in pivot table: actor_movie
    lst_actors = movie[10]
    fill_in_pivot_table('actor_movie', 'actor_id', lst_actors, movie_id, cursor)

    # Fill in pivot table: composer_movie
    lst_composers = movie[11]
    fill_in_pivot_table('composer_movie', 'composer_id', lst_composers, movie_id, cursor)
        
cnx.commit()

## **Quelques requêtes SQL**

**Requête** : Afficher les films dont le titre commence par "A" ainsi que les acteurs de ces films. 

In [39]:
query = (" \
SELECT m.title, a.actor_name \
FROM movies AS m \
JOIN actor_movie AS am ON am.movie_id = m.movie_id \
JOIN actors AS a ON a.actor_id = am.actor_id \
WHERE m.title LIKE 'A%' \
ORDER BY a.actor_name;")
cursor.execute(query)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=['title', 'actor'])
df

,title,actor
0,"Amityville 2, Le Possédé",Andrew Prine
1,"Amityville 2, Le Possédé",Brent Katz
2,"Amityville 2, Le Possédé",Burt Young
3,"Amityville 2, Le Possédé",Diane Franklin
4,"Amityville 2, Le Possédé",Jack Magner
5,"Amityville 2, Le Possédé",James Olson
6,"Amityville 2, Le Possédé",Leo Cimino
7,"Amityville 2, Le Possédé",Moses Gunn
8,"Amityville 2, Le Possédé",Rutanya Alda
9,"Amityville 2, Le Possédé",Ted Ross


**Requête** : Afficher les films dont le star rating est supérieur à 4.5. 

In [42]:
query = (" \
SELECT m.title \
FROM movies AS m \
WHERE m.star_rating > 4;")
cursor.execute(query)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=['title'])
df

,title
0,Gandhi
1,E.T. l'extra-terrestre
2,Le Père Noël est une ordure
3,Pink Floyd The Wall
4,Fanny et Alexandre - Partie 1
5,The Thing
6,Blade Runner
7,The Plague Dogs


**Requête** : Afficher le nom des acteurs qui ont joué dans plusieurs films. 

In [65]:
query = (" \
SELECT a.actor_name, COUNT(*) \
FROM movies AS m \
JOIN actor_movie AS am ON am.movie_id = m.movie_id \
JOIN actors AS a ON a.actor_id = am.actor_id \
GROUP BY a.actor_name \
HAVING COUNT(*) > 1 \
ORDER BY a.actor_name;")
cursor.execute(query)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=['actor', 'nb films'])
df

,actor,nb films
0,Adrienne Barbeau,2
1,Andre Chaumeau,2
2,Andréa Ferréol,3
3,Anémone,3
4,Annette O'Toole,2
...,...,...
84,Teri Garr,2
85,Thierry Lhermitte,2
86,William Shatner,2
87,Yann Dedet,2


**Requête** : Afficher le nombre d'acteurs ayant joué dans 1 film, puis 2 films etc ... 

In [117]:
query = (" \
WITH nb_films AS ( \
    SELECT a.actor_name AS name, COUNT(*) AS nb \
    FROM movies AS m \
    JOIN actor_movie AS am ON am.movie_id = m.movie_id \
    JOIN actors AS a ON a.actor_id = am.actor_id \
    GROUP BY a.actor_name \
    HAVING COUNT(*) > 0 \
    ORDER BY a.actor_name \
    ) \
SELECT DISTINCT nb, COUNT(*) \
FROM nb_films \
GROUP BY nb \
ORDER BY nb;")
cursor.execute(query)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=['nb films', "nb d'acteurs"])
df
# 914 + 2 * 74 + 3 * 14 + 6 = 1110  c'est le nombre d'acteurs en comptant les répétitions OK !!!

,nb films,nb d'acteurs
0,1,914
1,2,74
2,3,14
3,6,1


**Nous vérifions le nombre d'acteurs**

In [59]:
query = (" \
SELECT a.actor_name \
FROM movies AS m \
JOIN actor_movie AS am ON am.movie_id = m.movie_id \
JOIN actors AS a ON a.actor_id = am.actor_id \
ORDER BY a.actor_name;")
cursor.execute(query)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=['actor'])
df

,actor
0,Adam Redfield
1,Adrian Zmed
2,Adrienne Barbeau
3,Adrienne Barbeau
4,Al Ruban
...,...
1105,Yves Mourousi
1106,Yves Pignot
1107,Zabou Breitman
1108,Zabou Breitman
